In [ ]:
### attempting glm with python ###

In [52]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.genmod.bayes_mixed_glm as stat

In [15]:
df = pd.read_csv(r"E:\Speechgraphs_paper\same_size_5_year_all_sciences_thirds\final_table_non_retracted.csv", dtype={'Science': "category"})

In [16]:
df.drop(["L1", "L2", "L3", "ASP", "CC", "Edges", "Nodes", "PE", "LCC", "Diameter", "Density"], axis=1, inplace=True)

In [17]:
df["Valence"] = df["Valence"].apply(lambda x: 1 if x == "high" else 0)
df.Valence = pd.to_numeric(df.Valence)

In [18]:
df["id"] = df.reset_index().index

In [19]:
df.Date = df.Date.apply(lambda x: 0 if x == "80_85" else x)
df.Date = df.Date.apply(lambda x: 1 if x == "85_90" else x)
df.Date = df.Date.apply(lambda x: 2 if x == "90_95" else x)
df.Date = df.Date.apply(lambda x: 3 if x == "95_00" else x)
df.Date = df.Date.apply(lambda x: 4 if x == "00_05" else x)
df.Date = df.Date.apply(lambda x: 5 if x == "05_10" else x)
df.Date = df.Date.apply(lambda x: 6 if x == "10_15" else x)
df.Date = df.Date.apply(lambda x: 7 if x == "15_20" else x)

In [20]:
df.Date = df.Date.astype(int)

In [21]:
df.head(5)

,File,WC,RE,LSC,ATD,Science,Part,Valence,Date,retracted,id
0,bio_1_df_high_values_00_05_0,50,0,16,2.000000,bio,1,1,4,0,0
1,bio_1_df_high_values_00_05_1,35,1,25,2.344828,bio,1,1,4,0,1
2,bio_1_df_high_values_00_05_10,19,0,13,2.000000,bio,1,1,4,0,2
3,bio_1_df_high_values_00_05_100,27,0,17,2.166667,bio,1,1,4,0,3
4,bio_1_df_high_values_00_05_101,25,0,1,1.920000,bio,1,1,4,0,4


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151200 entries, 0 to 151199
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   File       151200 non-null  object  
 1   WC         151200 non-null  int64   
 2   RE         151200 non-null  int64   
 3   LSC        151200 non-null  int64   
 4   ATD        151200 non-null  float64 
 5   Science    151200 non-null  category
 6   Part       151200 non-null  int64   
 7   Valence    151200 non-null  int64   
 8   Date       151200 non-null  int32   
 9   retracted  151200 non-null  int64   
 10  id         151200 non-null  int64   
dtypes: category(1), float64(1), int32(1), int64(7), object(1)
memory usage: 11.1+ MB


In [38]:
# Assuming 'data' is your pandas DataFrame
result_list = []
for i in range(10000):

    boot_df = df.sample(1000, random_state=i)
    model_formula = "Valence ~ Date + WC + LSC + RE + Part + (Part|id)"
    model = stat.BinomialBayesMixedGLM(endog = boot_df.Valence, exog= boot_df[["Date", "WC", "LSC", "RE", "Part"]], group=boot_df.Science)
    result = model.fit()
    result_list.append(result)

In [50]:
print((result_list[5].pvalues))

<class 'pandas.core.series.Series'>


In [48]:
df_results = pd.DataFrame(columns=["p_intercept", "p_date", "p_wc", "p_LSC", "p_RE", "p_id", "p_group"])
for i in range(10000):
    df_results.loc[i] = result_list[i].pvalues

In [45]:
df_results.head()

,p_intercept,p_date,p_wc,p_LSC,p_RE,p_id,p_group
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
predicted_values = result.predict(new_data)